# K Nearest Neighbour
## Using MovieLens dataset to train and classify movies by genre and rating

In [1]:
import pandas as pd
import numpy as np

In [2]:
columns = ['user_id', 'movie_id', 'rating']
ratings = pd.read_csv("arquivos\\u.data", names=columns, usecols=range(3), sep='\t')

In [3]:
# ratings.head()

In [4]:
movieProperties = ratings.groupby('movie_id').agg({'rating':[np.size, 'mean']})
# movieProperties

In [5]:
movieNumRatings = movieProperties['rating']['size']
# movieNumRatings

Normalizing the number of ratings per movie

In [6]:
movieNumRatingsNormalized = movieNumRatings.apply(lambda movie: (movie - np.min(movieNumRatings))    /   (np.max(movieNumRatings)-np.min(movieNumRatings))    ).copy()
# movieNumRatingsNormalized

In [7]:
# len(movieNumRatingsNormalized)

In [8]:
genres = []
with open('arquivos\\u.genre') as f:
    for line in f:
        if line.strip():
            genres.append(line.split('|')[0])

In [9]:
fileImport = pd.read_csv('arquivos\\u.item', sep='|', names=(['movie_id', 'title'] + genres),usecols=[0,1] + list(range(5,24)), encoding='latin1')
# fileImport

In [10]:
movieDict = pd.DataFrame(columns=['Movie ID', 'Movie Name', 'Genre', 'Normalized Num of Ratings', 'Average Rating'])

In [95]:
movieDict['Movie ID'] = fileImport['movie_id']
movieDict['Movie Name'] = fileImport['title']
movieDict['Genre'] = fileImport.apply(axis=1, func= lambda x: x[2:].tolist())
movieDictID = movieDict.set_index('Movie ID')
movieDictID['Average Rating'] = movieNumRatings = movieProperties['rating']['mean']
movieDictID['Normalized Num of Ratings'] = movieNumRatingsNormalized
movieDictID.reset_index(inplace=True)

In [102]:
from scipy import spatial

# a e b passados como series
def ComputeDistance(a,b):
    genreA = a['Genre']
    genreB = b['Genre']
    genreDistance = spatial.distance.cosine(genreA, genreB)
    popularityA = a['Normalized Num of Ratings']
    popularityB = b['Normalized Num of Ratings']
    popularityDistance = abs(popularityA - popularityB)
    return genreDistance + popularityDistance


print(ComputeDistance(movieDictID.iloc[0], movieDictID.iloc[1]))
print(ComputeDistance(movieDictID.iloc[0], movieDictID.iloc[94]))
print(movieDictID.iloc[0])
print(movieDictID.iloc[1])
print(movieDictID.iloc[94])


1.5506003430531732
0.5336315430423195
Movie ID                                                                     1
Movie Name                                                    Toy Story (1995)
Genre                        [0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
Normalized Num of Ratings                                             0.773585
Average Rating                                                        3.878319
Name: 0, dtype: object
Movie ID                                                                     2
Movie Name                                                    GoldenEye (1995)
Genre                        [0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
Normalized Num of Ratings                                             0.222985
Average Rating                                                        3.206107
Name: 1, dtype: object
Movie ID                                                                    95
Movie Name                                     

In [113]:


def GetNeighbors(MovieID, K):
    distances = []
    distances = movieDictID.apply(func=lambda row: (row['Movie ID'],ComputeDistance(movieDictID.iloc[MovieID], row)), axis=1).tolist()
    distances.sort(key=lambda a: a[1])
    # print(distances)
    neighbors = []
    for i in range(1, K+1):
        # print(distances[i][0])
        neighbors.append(movieDictID.iloc[distances[i][0]-1])
    return neighbors



K = 10
avgRating = 0
neighbors = GetNeighbors(0,10)
for neighbor in neighbors:
    avgRating += neighbor['Average Rating']
    print(neighbor['Movie Name'] + ' ' + str(neighbor['Average Rating']))
# movieDictID.apply(func=lambda row: (row,ComputeDistance(movieDictID.loc[1], row)), axis=1)
avgRating / K
# ComputeDistance(movieDictID.loc[1], movieDictID.loc[2])

Liar Liar (1997) 3.156701030927835
Aladdin (1992) 3.8127853881278537
Willy Wonka and the Chocolate Factory (1971) 3.6319018404907975
Monty Python and the Holy Grail (1974) 4.0664556962025316
Full Monty, The (1997) 3.926984126984127
George of the Jungle (1997) 2.685185185185185
Beavis and Butt-head Do America (1996) 2.7884615384615383
Birdcage, The (1996) 3.4436860068259385
Home Alone (1990) 3.0875912408759123
Aladdin and the King of Thieves (1996) 2.8461538461538463


3.3445905900235564